In [4]:
#Installs the Poliastro librabry in order to use its lambert solver
!pip install poliastro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.0 MB/s eta 0:00:00


In [5]:
#Importing all packages used for the code
import numpy as np
from astropy import units as u
from poliastro.bodies import Earth
from poliastro.iod import izzo
import random

# A function that, when called, returns a (1x3) array in which each position contains a uniformly randomized integer between -10000 and 10000 in poliasto kilometer units
def random_position():
    return np.array([random.uniform(-10000, 10000), random.uniform(-10000, 10000), random.uniform(-10000, 10000)]) * u.km

# Optimum loop function
#Four inputs: r0 represents the initial postion vector, rf represents the final position vector, and tof is the time of flight.
#Max_revs represents the maximum number of revolutions the lambert solver can loop to. The function returns the optimum solution(initial and velocity vectors)
#The function will also return the optimal_num_revs which is the revolution related to the solution that produced the lowest delta-V
def find_optimal_solution(r0, rf, tof, max_revs):
    #Initializing returning values
    min_delta_v = float("inf")
    optimal_solution = None
    optimal_num_revs = None

    #Loop through all revolutions to the max values and solve for each solution
    for num_revs in range(max_revs + 1):
        #I use a try/except method so that the code may keep executing until the function fails to converge then just outputs the optimal solution
        try:
            v0, vf = izzo.lambert(Earth.k, r0, rf, tof, M=num_revs)
            delta_v = np.linalg.norm(vf - v0)

            #Compare the the currents solution delta-V to the stored lowest delta-V. If smaller replace as the most optimum solution and save the values.
            if delta_v < min_delta_v:
                min_delta_v = delta_v
                optimal_solution = (v0, vf)
                optimal_num_revs = num_revs
        except ValueError:
            pass

    return optimal_solution, optimal_num_revs

#Creates the simulated data array with zeros
results = np.zeros((100000, 14))

#Create the 100,000 data points
for i in range(100000):

  #Initialize random position vectors
  r0 = random_position()
  rf = random_position()

  #Creates a random tof from 30 minutes to 10,080 minutes or one week.
  tof = random.uniform(30, 10080) * u.min # 1 Week

  #Limits the find_optimal_solution search in order to reduce computation time
  max_revs = 20

  #Calls the find_optimal solution function
  optimal_solution, optimal_num_revs = find_optimal_solution(r0, rf, tof, max_revs)
  v0, vf = optimal_solution

  #Store results and inputs into one data point.
  result = np.hstack((
        r0.value,
        rf.value,
        tof.value,
        v0.value,
        vf.value,
        optimal_num_revs))

  results[i, :] = result

#Saves the data into a csv file to be later used 
np.savetxt("lambert_data.csv", results, delimiter=",", header="r0_x,r0_y,r0_z,rf_x,rf_y,rf_z,tof,v0_x,v0_y,v0_z,vf_x,vf_y,vf_z,num_revs")

Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x7f6d881655e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/llvmlite/binding/executionengine.py", line 171, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 


KeyboardInterrupt: ignored